# OT2

This notebook shows how to use the Opentrons OT2 with Pylabrobot.

```{note}
The 8 channel pipette is not yet supported.
```

## Connecting

There are different ways to find the robot IP address.
- the OT vendor app.
- using its hostname, e.g. `ot2.local`.
- `arp -a` command in terminal to list all devices in the network.
- checking the connected devices in your router admin panel.
- ...

In [1]:
from pylabrobot.liquid_handling import LiquidHandler, OpentronsOT2Backend
from pylabrobot.resources import OTDeck
deck = OTDeck()
robot_ip = "192.168.0.61"
ot2_backend = OpentronsOT2Backend(host=robot_ip)
lh = LiquidHandler(backend=ot2_backend, deck=deck)

You can set the traversal height for the robot as follows. This is the height above the deck at which tips will move.

In [2]:
ot2_backend.traversal_height = 120

The robot will automatically home on `setup`, unless `skip_home=True` is passed.

In [3]:
await lh.setup(skip_home=False)

Homing manually:

In [4]:
await ot2_backend.home()

## Deck setup

Loading a tip rack from the OT reosurce library. This requires a network connection the first time to download the labware definition. After that it is cached locally.

In [5]:
from pylabrobot.resources.opentrons.load import load_ot_tip_rack
tr20uL = load_ot_tip_rack(
  ot_name="opentrons_96_filtertiprack_20ul",  # https://labware.opentrons.com
  plr_resource_name="20uL Tip Rack 1"  # name used in Pylabrobot, arbitrary
)
deck.assign_child_at_slot(tr20uL, slot=9)

Plates are loaded from [the PLR resource library](https://docs.pylabrobot.org/resources/index.html) as usual.

In [6]:
from pylabrobot.resources import agilent_96_wellplate_150uL_Ub
plate = agilent_96_wellplate_150uL_Ub(name="premixplate")
deck.assign_child_at_slot(plate, slot=6)

You can also import tube racks from the OT resource library and use them with normal PLR tubes:

In [7]:
from pylabrobot.resources.opentrons.load import load_ot_tube_rack
from pylabrobot.resources import Eppendorf_DNA_LoBind_1_5ml_Vb
tube_rack = load_ot_tube_rack(
  ot_name="opentrons_24_aluminumblock_nest_1.5ml_snapcap",
  plr_resource_name="tube_rack"
)
tube_rack[0] = tube = Eppendorf_DNA_LoBind_1_5ml_Vb(name="tube_01")
deck.assign_child_at_slot(tube_rack, slot=4)

## Liquid handling

OT2 robots often get out of calibration or do not calibrate to the deck correctly.

You can deal with this by passing offsets to specific operations. The offsets are in mm. Below shows 0,0,0 but you might need a mm to make operations work on your robot. Be careful that the offsets will drift over time.

In [8]:
from pylabrobot.resources.coordinate import Coordinate
tip_offset = Coordinate(x=0, y=0, z=0)
await lh.pick_up_tips(tr20uL[95], offsets=[tip_offset])

Moving liquid from a plate to a tube:

In [9]:
plate_offset = Coordinate(x=0, y=0, z=0)
await lh.aspirate(plate[0], vols=[20], offsets=[plate_offset])
tube_offset = Coordinate(x=0, y=0, z=0)
await lh.dispense([tube], vols=[20], offsets=[tube_offset])

You can put the tips in the trash after use using `await lh.discard_tips()`.

In [10]:
trash_tip = False
if trash_tip:
  await lh.discard_tips()

Or return them to the pickup location using `await lh.return_tips()`.

In [11]:
await lh.return_tips(offsets=[tip_offset])